<a href="https://colab.research.google.com/github/paulis-an/sql-course/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22sql_tests_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
6.23MB [00:00, 93.2MB/s]


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import sqlite3

In [ ]:
sqlite3.sqlite_version

'3.34.1'

In [ ]:
con = sqlite3.connect('db')

In [ ]:
!gdown --id 1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR

Downloading...
From: https://drive.google.com/uc?id=1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR
To: /content/german_credit_augmented.csv
100% 69.6k/69.6k [00:00<00:00, 24.6MB/s]


In [ ]:
df = pd.read_csv('/content/german_credit_augmented.csv')

In [ ]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [ ]:
df['contract_dt'] = pd.to_datetime(df['contract_dt'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
df.dtypes

age                          int64
sex                         object
job                          int64
housing                     object
saving_accounts             object
checking_account            object
credit_amount                int64
duration                     int64
purpose                     object
default                      int64
contract_dt         datetime64[ns]
client_id                    int64
dtype: object

In [ ]:
df.to_sql('german_credit',con,index=False,if_exists='replace')

In [ ]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [ ]:
def select(sql):
  return pd.read_sql(sql,con)


In [ ]:
sql = '''select
 t.age * 3 as age_mult3,
 t.sex from german_credit t limit 5'''

In [ ]:
select(sql)

,age_mult3,sex
0,99,male
1,129,male
2,156,male
3,105,female
4,84,male


In [ ]:
sql = '''select count(*) from german_credit t 
        where t.contract_dt between '2007-01-01' and '2007-12-31' and
        t.purpose = 'car'
        order by contract_dt'''

In [ ]:
select(sql)

In [ ]:
sql = '''select * from german_credit t 
        where t.contract_dt between '2007-01-01' and '2007-12-31' 
        --and t.purpose = 'car'
        order by contract_dt
        limit 10'''

In [ ]:
select(sql)

In [ ]:
sql = '''select * from german_credit t 
        where t.contract_dt between '2007-01-01' and '2007-12-31' 
        and t.purpose in('car', 'repairs')
        order by contract_dt
        '''

In [ ]:
select(sql)

In [ ]:
t = pd.DataFrame({'purpose':['машина','на машину', 'на покупку машины', 'автомобиль', 'на возвращение кредита 2007'],
              'amount':[1000, 400, 600, 700, 1500]})

In [ ]:
t

In [ ]:
t.to_sql('purpose', con, index=False, if_exists="replace")

In [ ]:
sql = ''' select * from purpose t
where (t.purpose like '%машин%'
or t.purpose like '%авто%')
and t.amount > 600
'''

In [ ]:
select(sql)

In [ ]:
sql = '''select count(*) from german_credit t 
where t.credit_amount > 1000
'''

In [ ]:
select(sql)

In [ ]:
sql = '''select t.credit_amount, 
case when t.credit_amount > 1000 then 1 else 0 end as greater_1000_flag,
iif(t.credit_amount > 1000, 1, 0) as greater_1000_flag2
from german_credit t;
'''

In [ ]:
select(sql)

In [ ]:
sql = '''select 
avg(case when t.credit_amount > 1000 then 1 else 0 end) as greater_1000_frac
from german_credit t;
'''

In [ ]:
select(sql)

Создание таблицы из запроса


In [ ]:
cur = con.cursor()

In [ ]:
sql = '''
drop table if exists greater_1000_credit;

create table greater_1000_credit as
select *
from german_credit t;
'''

In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''
select * from greater_1000_credit t
where t.credit_amount > 1000
'''

In [ ]:
select(sql)

UNION ALL

In [ ]:
jan = pd.DataFrame({'month':['jan', 'jan'], 'revenue':[1,2]})
feb = pd.DataFrame({'month':['feb', 'feb'], 'revenue':[1,2]})

In [ ]:
jan.to_sql('jan', con, index = False, if_exists='replace')
feb.to_sql('feb', con, index = False, if_exists='replace')

In [ ]:
sql = '''
select * from jan t
'''

In [ ]:
select(sql)

In [ ]:
sql = '''
select * from jan t
UNION ALL
select * from feb
'''

In [ ]:
select(sql)

In [ ]:
sql = '''
select t.revenue, 'jan' as month
from jan t
UNION ALL
select feb.revenue, 'feb' as month
from feb
'''

In [ ]:
select(sql)

GROUP BY

In [ ]:
sql = '''
select t.sex,
count(*) as count,
avg(t.credit_amount) as avg_credit
from german_credit t
group by t.sex
'''

In [ ]:
select(sql)

In [ ]:
sql = '''
select t.housing,
count(*) as count,
avg(t.credit_amount) as avg_credit
from german_credit t
group by t.housing
'''

In [ ]:
select(sql)

In [ ]:
sql='''
select count(t.checking_account), count(*) from german_credit t
'''

In [ ]:
select(sql)

In [ ]:
sql='''
select t.checking_account, count(t.checking_account), count(*), (count(*) - count(t.checking_account)) as None_count from german_credit t
group by t.checking_account
'''

In [ ]:
select(sql)

,checking_account,count(t.checking_account),count(*),None_count
0,None,0,394,394
1,little,274,274,0
2,moderate,269,269,0
3,rich,63,63,0


In [ ]:
sql='''
select 
sum(case when t.checking_account is null then 1 else 0 end) as is_null,
count(case when t.checking_account is null then 1 else null end) as is_null2 from german_credit t
'''

In [ ]:
select(sql)

,is_null,is_null2
0,394,394


In [ ]:
sql = '''
select date(t.contract_dt, 'start of month')  as month, count(*) as credit_cnt,
count(distinct t.client_id) as client_id_unique,
sum(t.credit_amount) as sum, round(avg(t.credit_amount), 2) as avg
from german_credit t
group by 1
order by 1
'''

In [ ]:
select(sql)

,month,credit_cnt,client_id_unique,sum,avg
0,2007-05-01,81,81,207663,2563.74
1,2007-06-01,74,74,239594,3237.76
2,2007-07-01,71,71,224333,3159.62
3,2007-08-01,57,57,178569,3132.79
4,2007-09-01,58,58,186909,3222.57
5,2007-10-01,70,70,188534,2693.34
6,2007-11-01,87,87,300504,3454.07
7,2007-12-01,77,77,273973,3558.09
8,2008-01-01,93,93,288080,3097.63
9,2008-02-01,55,55,211128,3838.69


In [ ]:
sql = '''
select count(distinct t.credit_amount) as cridit_amount_unique from german_credit t '''

In [ ]:
select(sql)

,cridit_amount_unique
0,921


In [ ]:
sql = '''
select t.credit_amount,
case  when t.credit_amount < 1000 then '1. <1000' 
      when t.credit_amount < 2000 then '2. 1000-2000'
      when t.credit_amount < 3000 then '3. 2000-3000'
      when t.credit_amount >= 3000 then '4. >= 3000'
      else 'other' end as credit_amount_bin
 from german_credit t '''

In [ ]:
select(sql)

,credit_amount,credit_amount_bin
0,3074,4. >= 3000
1,1344,2. 1000-2000
2,936,1. <1000
3,1393,2. 1000-2000
4,776,1. <1000
...,...,...
995,2600,3. 2000-3000
996,4455,4. >= 3000
997,6403,4. >= 3000
998,5003,4. >= 3000


In [ ]:
sql = '''
select 
case  when t.credit_amount < 1000 then '1. <1000' 
      when t.credit_amount < 2000 then '2. 1000-2000'
      when t.credit_amount < 3000 then '3. 2000-3000'
      when t.credit_amount >= 3000 then '4. >= 3000'
      else 'other' end as credit_amount_bin, 
count(*) as credit_count
from german_credit t 
group by 1
'''

In [ ]:
select(sql)

,credit_amount_bin,credit_count
0,1. <1000,116
1,2. 1000-2000,316
2,3. 2000-3000,188
3,4. >= 3000,380


Перменнные в столбцах сводной таблицы

In [ ]:
sql = '''
select t.housing, 
  count(case when t.sex = 'female' then 1 else null end) as female,
  count(case when t.sex = 'male' then 1 else null end) as male,
  count(*) as cnt
  from german_credit t
  group by t.housing
'''

In [ ]:
select(sql)

,housing,female,male,cnt
0,free,19,89,108
1,own,196,517,713
2,rent,95,84,179


In [ ]:
sql = ''' select distinct t.purpose from german_credit t'''

In [ ]:
purpose = list(select(sql)['purpose'].values)

In [ ]:
purpose

['radio/TV',
 'car',
 'education',
 'furniture/equipment',
 'repairs',
 'business',
 'domestic appliances',
 'vacation/others']

In [ ]:
for p in purpose:
  print(f"count(case when t.purpose = '{p}' then 1 else null end) as {p.lower().replace(' ','_').replace('/','_')},")

count(case when t.purpose = 'radio/TV' then 1 else null end) as radio_tv,
count(case when t.purpose = 'car' then 1 else null end) as car,
count(case when t.purpose = 'education' then 1 else null end) as education,
count(case when t.purpose = 'furniture/equipment' then 1 else null end) as furniture_equipment,
count(case when t.purpose = 'repairs' then 1 else null end) as repairs,
count(case when t.purpose = 'business' then 1 else null end) as business,
count(case when t.purpose = 'domestic appliances' then 1 else null end) as domestic_appliances,
count(case when t.purpose = 'vacation/others' then 1 else null end) as vacation_others,


In [ ]:
sql = '''
select t.housing, 
  count(case when t.purpose = 'radio/TV' then 1 else null end) as radio_tv,
count(case when t.purpose = 'car' then 1 else null end) as car,
count(case when t.purpose = 'education' then 1 else null end) as education,
count(case when t.purpose = 'furniture/equipment' then 1 else null end) as furniture_equipment,
count(case when t.purpose = 'repairs' then 1 else null end) as repairs,
count(case when t.purpose = 'business' then 1 else null end) as business,
count(case when t.purpose = 'domestic appliances' then 1 else null end) as domestic_appliances,
count(case when t.purpose = 'vacation/others' then 1 else null end) as vacation_others,
  count(*) as cnt
  from german_credit t
  group by t.housing
'''

In [ ]:
select(sql)

,housing,radio_tv,car,education,furniture_equipment,repairs,business,domestic_appliances,vacation_others,cnt
0,free,15,55,15,11,3,5,0,4,108
1,own,227,219,34,122,17,76,10,8,713
2,rent,38,63,10,48,2,16,2,0,179
